In [ ]:
from influx_kwh import InfluxTempReader, format_entity_id
from tqdm import tqdm
import pandas as pd
pd.options.plotting.backend = "plotly"
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import datetime
from dateutil.relativedelta import relativedelta
import numpy as np
import pytz
import math

In [ ]:
start_date = datetime(2024, 5, 2,0, 0, 0).replace(tzinfo=pytz.UTC)
end_date = datetime.today().replace(tzinfo=pytz.UTC)

host=''
port=8086
username = ''
password = ''
dbname = ''

est = pytz.timezone('US/Eastern')

In [ ]:
temp_reader = InfluxTempReader(host, port, username, password, dbname)
nb_months = (end_date - start_date).days / 30
import_ranges = [(start_date + relativedelta(days=30 * offset), start_date + relativedelta(days=30 * (offset+1))) for offset in range(math.ceil(nb_months))]
df_stack = pd.concat([temp_reader.query_data(s, e) for (s,e) in tqdm(import_ranges)])
df_stack.index = df_stack.index.tz_convert(est)

In [ ]:
columns = [c for c in df_stack.columns if not c.startswith('temperature_sensor') and not '_t_' in c and not 'thermostat' in c and not 'Thermostat' in c]
df_stack = df_stack[columns][df_stack.index > '2024-05-03'].dropna(how='all').rename(columns=format_entity_id)

In [ ]:
cols = [c for c in df_stack.columns if c != 'Aqara Temp Humidity Sensor']
df_stack = df_stack[cols]

In [ ]:
df_stack

In [ ]:
df_stack.resample('H').mean().plot()

In [ ]:
# Import Meteostat library and dependencies
from datetime import datetime
from meteostat import Point, Hourly, units

start = datetime(2024, 5, 2,0, 0, 0)
end = datetime.today()

location = Point(26.077885, -80.321570)

data = Hourly(location, start, end)
data = data.convert(units.imperial)
data = data.fetch()

In [ ]:
data.index = [d.replace(tzinfo=pytz.UTC).astimezone(est) for d in data.index]
df_stack['meteo'] = data['temp']
df_stack.meteo = df_stack.meteo.ffill().bfill()

In [ ]:
df_stack[['Pool Temp', 'meteo']].plot()

In [ ]:
df_rolling_mean_diff = df_stack.rolling(30).apply(np.mean).diff()

In [ ]:
group1 = []
group2 = []
group3 = []
group4 = []

columns = group1 + group2 + group3 + group4

In [ ]:
df_rolling_mean_diff[columns].corr().style.background_gradient(axis=None) 